In [49]:
! pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import tensorflow as tf


In [51]:
! pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [53]:
! pip install tf-keras


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [55]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bindu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bindu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
# Loading the dataset
df = pd.read_csv('fake_job_postings.csv')

In [58]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Return an empty string for non-string values
    text = text.lower()  # Lowercase text
    text = re.sub(r'http\S+', '', text)  # Remove links
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words containing numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    text = ' '.join(words)
    return text

df['text'] = df['description'].apply(preprocess_text)

# Stemming and Lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
    words = text.split()
    stemmed = [stemmer.stem(word) for word in words]
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return ' '.join(lemmatized)

df['text'] = df['text'].apply(stem_and_lemmatize)

# Check for missing values
df.dropna(subset=['text'], inplace=True)

# Convert target variable to binary
df['target'] = df['fraudulent'].apply(lambda x: 1 if x == 'yes' else 0)


In [63]:
# Split the dataset
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(temp_data, test_size=0.33, random_state=42)


In [64]:

# Extract features and labels
X_train = train_data['text']
y_train = train_data['target']
X_test = test_data['text']
y_test = test_data['target']
X_val = val_data['text']
y_val = val_data['target']


In [65]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
X_val_tfidf = vectorizer.transform(X_val)

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
# XGBoost Model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [68]:

# Predictions and evaluation
y_pred_train_xgb = xgb_model.predict(X_train_tfidf)
y_pred_test_xgb = xgb_model.predict(X_test_tfidf)

In [69]:
# Calculate accuracy
train_accuracy_xgb = accuracy_score(y_train, y_pred_train_xgb)
test_accuracy_xgb = accuracy_score(y_test, y_pred_test_xgb)


In [70]:

# Print accuracy
print("XGBoost - Training Accuracy:", train_accuracy_xgb)
print("XGBoost - Testing Accuracy:", test_accuracy_xgb)

XGBoost - Training Accuracy: 1.0
XGBoost - Testing Accuracy: 1.0


In [71]:

# Classification report for detailed performance metrics
print("XGBoost - Training Classification Report")
print(classification_report(y_train, y_pred_train_xgb))
print("XGBoost - Testing Classification Report")
print(classification_report(y_test, y_pred_test_xgb))

XGBoost - Training Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12516

    accuracy                           1.00     12516
   macro avg       1.00      1.00      1.00     12516
weighted avg       1.00      1.00      1.00     12516

XGBoost - Testing Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3593

    accuracy                           1.00      3593
   macro avg       1.00      1.00      1.00      3593
weighted avg       1.00      1.00      1.00      3593



In [72]:

# Hyperparameter Tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

In [74]:
grid_xgb = GridSearchCV(xgb.XGBClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_xgb.fit(X_train_tfidf, y_train)



c:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
253 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-pack

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 150],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [75]:

best_xgb = grid_xgb.best_estimator_

In [76]:
# Evaluation of best model
y_pred_train_best_xgb = best_xgb.predict(X_train_tfidf)
y_pred_test_best_xgb = best_xgb.predict(X_test_tfidf)

In [77]:
# Calculate accuracy
train_accuracy_best_xgb = accuracy_score(y_train, y_pred_train_best_xgb)
test_accuracy_best_xgb = accuracy_score(y_test, y_pred_test_best_xgb)


In [78]:
# Print accuracy
print("Best XGBoost - Training Accuracy:", train_accuracy_best_xgb)
print("Best XGBoost - Testing Accuracy:", test_accuracy_best_xgb)


Best XGBoost - Training Accuracy: 1.0
Best XGBoost - Testing Accuracy: 1.0


In [79]:
# Classification report for detailed performance metrics
print("Best XGBoost - Training Classification Report")
print(classification_report(y_train, y_pred_train_best_xgb))
print("Best XGBoost - Testing Classification Report")
print(classification_report(y_test, y_pred_test_best_xgb))

Best XGBoost - Training Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12516

    accuracy                           1.00     12516
   macro avg       1.00      1.00      1.00     12516
weighted avg       1.00      1.00      1.00     12516

Best XGBoost - Testing Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3593

    accuracy                           1.00      3593
   macro avg       1.00      1.00      1.00      3593
weighted avg       1.00      1.00      1.00      3593



LSTM MODEL

In [83]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping


In [84]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)


In [85]:
maxlen = 100  # Set maximum sequence length

In [86]:
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)
X_val_pad = pad_sequences(X_val_seq, maxlen=maxlen)

In [87]:
# LSTM Model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))  # Removed input_length argument
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

In [88]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



In [89]:
# Train the model
history = model.fit(
    X_train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_pad, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 95s 172ms/step - accuracy: 0.9885 - loss: 0.0637 - val_accuracy: 1.0000 - val_loss: 5.7090e-05
Epoch 2/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 59s 150ms/step - accuracy: 1.0000 - loss: 5.1040e-05 - val_accuracy: 1.0000 - val_loss: 2.6750e-05
Epoch 3/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 58s 147ms/step - accuracy: 1.0000 - loss: 2.5311e-05 - val_accuracy: 1.0000 - val_loss: 1.5982e-05
Epoch 4/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - accuracy: 1.0000 - loss: 1.5442e-05 - val_accuracy: 1.0000 - val_loss: 1.0608e-05
Epoch 5/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 57s 146ms/step - accuracy: 1.0000 - loss: 1.0363e-05 - val_accuracy: 1.0000 - val_loss: 7.4780e-06
Epoch 6/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 64s 163ms/step - accuracy: 1.0000 - loss: 7.3245e-06 - val_accuracy: 1.0000 - val_loss: 5.4692e-06
Epoch 7/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 59s 149ms/step - accuracy: 1.0000 - loss: 5.3834e-06 - val_accuracy: 1.0000 - val_loss: 4.0626e-06
Epoch 8/10
392/392 ━━━━━━━━━━━━

In [90]:
# Predictions and evaluation
y_pred_train_lstm = (model.predict(X_train_pad) > 0.5).astype("int32")
y_pred_test_lstm = (model.predict(X_test_pad) > 0.5).astype("int32")

392/392 ━━━━━━━━━━━━━━━━━━━━ 22s 52ms/step
113/113 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step


In [91]:
# Calculate accuracy
train_accuracy_lstm = accuracy_score(y_train, y_pred_train_lstm)
test_accuracy_lstm = accuracy_score(y_test, y_pred_test_lstm)

In [92]:
# Print accuracy
print("LSTM - Training Accuracy:", train_accuracy_lstm)
print("LSTM - Testing Accuracy:", test_accuracy_lstm)

LSTM - Training Accuracy: 1.0
LSTM - Testing Accuracy: 1.0


In [93]:
# Classification report for detailed performance metrics
print("LSTM - Training Classification Report")
print(classification_report(y_train, y_pred_train_lstm))
print("LSTM - Testing Classification Report")
print(classification_report(y_test, y_pred_test_lstm))

LSTM - Training Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12516

    accuracy                           1.00     12516
   macro avg       1.00      1.00      1.00     12516
weighted avg       1.00      1.00      1.00     12516

LSTM - Testing Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3593

    accuracy                           1.00      3593
   macro avg       1.00      1.00      1.00      3593
weighted avg       1.00      1.00      1.00      3593



BERT BASED MODEL

In [117]:
! pip install torch


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
! pip install transformers torch

   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.4/159.8 MB 8.5 MB/s eta 0:00:19
   ---------------------------------------- 1.7/159.8 MB 17.9 MB/s eta 0:00:09
    --------------------------------------- 2.8/159.8 MB 19.8 MB/s eta 0:00:08
    --------------------------------------- 3.4/159.8 MB 18.2 MB/s eta 0:00:09
   - -------------------------------------- 4.0/159.8 MB 17.1 MB/s eta 0:00:10
   - -------------------------------------- 4.6/159.8 MB 16.2 MB/s eta 0:00:10
   - -------------------------------------- 5.2/159.8 MB 16.5 MB/s eta 0:00:10
   - -------------------------------------- 5.8/159.8 MB 16.0 MB/s eta 0:00:10
   - -------------------------------------- 6.3/159.8 MB 15.5 MB/s eta 0:00:10
   - -------------------------------------- 6.9/159.8 MB 15.3 MB/s eta 0:00:11
   - -------------------------------------- 7.5/159.8 MB 15.0 MB/s eta 0:00:11
   -- ------------------------------------- 8.1/159.8 MB 14.7


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [98]:

# Split the dataset
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(temp_data, test_size=0.33, random_state=42)

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [99]:

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [103]:
! pip install datasets

   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   ----------------------------- ---------- 399.4/547.8 kB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 547.8/547.8 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/316.1 kB ? eta -:--:--
   ------------------------------- -------- 245.8/316.1 kB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 316.1/316.1 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/370.8 kB ? eta -:--:--
   --------------------------------------- 370.8/370.8 kB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.9 MB 26.4 MB/s eta 0:00:01
   --- ------------------------------------ 2.4/25.9 MB 31.0 MB/s eta 0:00:01
   ---- ---------


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [104]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [105]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data[['text', 'target']])
test_dataset = Dataset.from_pandas(test_data[['text', 'target']])
val_dataset = Dataset.from_pandas(val_data[['text', 'target']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1771/1771 [00:06<00:00, 291.19 examples/s]


In [106]:

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'target'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'target'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'target'])

In [108]:
! pip install torch torchvision torchaudio


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.2 MB 4.6 MB/s eta 0:00:01
   ---------------------- ----------------- 0.7/1.2 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ------------------------ --------------- 1.4/2.4 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------  2.4/2.4 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 16.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [111]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import tensorflow as tf

In [114]:

# Convert datasets to tf.data.Dataset
train_features = {x: train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_dataset["target"])).batch(16)

In [115]:

val_features = {x: val_dataset[x] for x in tokenizer.model_input_names}
val_tf_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_dataset["target"])).batch(16)

test_features = {x: test_dataset[x] for x in tokenizer.model_input_names}
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_dataset["target"])).batch(16)

In [120]:
! pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [122]:
from transformers import BertTokenizer


In [123]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
train_encodings = tokenizer(train_data['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data['text'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data['text'].tolist(), truncation=True, padding=True)


In [124]:
import tensorflow as tf

# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_data['target'].tolist()
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_data['target'].tolist()
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_data['target'].tolist()
))

# Batch and shuffle datasets
train_dataset = train_dataset.shuffle(len(train_data)).batch(16)
val_dataset = val_dataset.batch(16)
test_dataset = test_dataset.batch(16)

In [126]:
! pip show tensorflow transformers


Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: tensorflow-intel
Required-by: tf_keras
---
Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\Bindu\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [127]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [128]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [129]:

# Example dataset (replace with your actual dataset loading)
train_texts = ["Example text 1", "Example text 2"]
train_labels = [0, 1]  # Binary labels (0 or 1)

In [125]:
from transformers import TFBertForSequenceClassification

# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.SparseCategoricalAccuracy()

# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    batch_size=16
)

TypeError: unsupported format string passed to SymbolicTensor.__format__

In [121]:
# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

TypeError: unsupported format string passed to SymbolicTensor.__format__